In [1]:
from unicodedata import numeric
import pandas as pd
import numpy as np
from tqdm import tqdm
import re

In [2]:
df = pd.read_csv('data/small_data.csv')
df['time'] = pd.to_datetime(df['time'])
df = df.set_index('time')

In [3]:
for col in df.columns:
    if (df[col] == 0).all():
        print(col)

FromInternetTCPPort443IPurn:ietf:params:mud:controllerByteCount
FromInternetTCPPort443IPurn:ietf:params:mud:controllerPacketCount
ToInternetTCPPort443IPurn:ietf:params:mud:controllerByteCount
ToInternetTCPPort443IPurn:ietf:params:mud:controllerPacketCount


In [4]:
def nonzero_proportion(x):
    return (x!=0).mean()

In [5]:
numeric_cols = list(df.columns)[:-1]
funcs = {'mean':np.mean, 'std':np.std, 'max':np.max, 'min':np.min, 'nonzero_p':nonzero_proportion}

In [6]:
df['time_group'] = df.index.to_period('H').strftime('%d_%H')

In [7]:
summary = {key: [] for key in funcs.keys()}
summary['info'] = []
summary['target'] = []

In [8]:
for time_slice in tqdm(df['time_group'].unique()):
    for device in (df['device_mac'].unique()):
        tmp = df.loc[(df['time_group']==time_slice)&(df['device_mac']==device)][numeric_cols]
        for col in tmp.columns:
            for k in funcs.keys():
                summary[k].append(funcs[k](tmp[col]))
            summary['info'].append(device+'_'+time_slice)
            summary['target'].append(col)

100%|██████████| 337/337 [01:14<00:00,  4.52it/s]


In [9]:
summary = pd.DataFrame(summary)
summary.to_csv('data/transformed_data.csv',index=False)

In [10]:
summary

,mean,std,max,min,nonzero_p,info,target
0,0.000000,0.000000,0,0,0.000000,bd:0d:8e:c5_02_00,FromInternetTCPPort443IPurn:ietf:params:mud:co...
1,0.000000,0.000000,0,0,0.000000,bd:0d:8e:c5_02_00,FromInternetTCPPort443IPurn:ietf:params:mud:co...
2,0.000000,0.000000,0,0,0.000000,bd:0d:8e:c5_02_00,FromInternetTCPPort554IPurn:ietf:params:mud:co...
3,0.000000,0.000000,0,0,0.000000,bd:0d:8e:c5_02_00,FromInternetTCPPort554IPurn:ietf:params:mud:co...
4,0.000000,0.000000,0,0,0.000000,bd:0d:8e:c5_02_00,FromInternetTCPPort80IPurn:ietf:params:mud:con...
...,...,...,...,...,...,...,...
29651,0.000000,0.000000,0,0,0.000000,ee:dd:34:1d_16_00,ToInternetUDPPort1024IPurn:ietf:params:mud:con...
29652,0.000000,0.000000,0,0,0.000000,ee:dd:34:1d_16_00,ToLocalUDPPort5353IP224.0.0.251/32ByteCount
29653,0.000000,0.000000,0,0,0.000000,ee:dd:34:1d_16_00,ToLocalUDPPort5353IP224.0.0.251/32PacketCount
29654,157.565217,343.405430,906,0,0.173913,ee:dd:34:1d_16_00,ToLocalUDPPort67IP255.255.255.255/32ByteCount
